In [8]:
# Import required libraries and dependencies
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import hvplot.pandas
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno 

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, log_loss
from tabulate import tabulate


# 1. Loading the Dataset from CSV

In [3]:
# Load the data into a Pandas DataFrame
df5 = pd.read_csv("heart_2022_cleaned.csv")
# Display sample data
df5

,Sex,Age,Race,BMI,GenHealth,PhysicalHealth,MentalHealth,PhysicalActivity,ChestScan,SleepHours,...,ECigaretteUsage,Drinking,HeartAttack,Angina,Stroke,Asthma,SkinCancer,Depressed,KidneyDisease,Diabetes
0,0,12,0,26.57,4,0.0,0.0,0,0,6.0,...,0,0,0,0,0,0,0,0,0,0
1,0,7,0,25.61,3,2.0,3.0,1,0,5.0,...,0,0,0,0,0,0,0,0,0,0
2,0,4,0,21.77,1,2.0,0.0,1,1,9.0,...,0,1,0,0,0,0,1,0,0,0
3,1,12,0,26.08,0,1.0,0.0,0,0,7.0,...,0,0,1,0,1,0,1,0,0,1
4,0,12,1,22.96,3,0.0,0.0,1,0,7.0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318929,1,9,3,29.86,2,0.0,15.0,1,0,7.0,...,0,1,0,0,1,0,1,0,0,1
318930,1,2,0,31.19,2,0.0,0.0,1,0,8.0,...,0,1,0,0,0,0,1,0,0,0
318931,0,6,1,28.66,4,2.0,2.0,1,0,7.0,...,0,0,0,0,0,0,1,0,0,0
318932,1,10,1,32.55,3,0.0,0.0,0,1,5.0,...,0,0,1,0,0,1,1,0,0,0


# 5. Machine Learning

## 5.1. Over Sampling

In [4]:
# Define features set X
X = df5.drop("HeartAttack", axis=1)
# Define target vector y
y = df5["HeartAttack"].values

##### Using MinMaxScaler() and SMOTE to deal with Imbalanced Data

In [5]:
# Dealing with Imbalanced Data. Scale features to [0, 1] range
X_scaled = MinMaxScaler().fit_transform(X)

# SMOTE for Resampling to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

#### Split the Resampled data into Training, Validation and Test sets. The validation set is used for tuning hyperparameters and early stopping.

In [6]:
# Split the data into training, validation, and test using train_test_split with stratification
X_train, X_temp, y_train, y_temp = train_test_split(
    X_resampled, 
    y_resampled, 
    stratify=y_resampled,  # to maintain the same proportion of classes in both train and test sets
    test_size=0.3,
    random_state=78
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    stratify=y_temp,
    test_size=0.5,
    random_state=78
)

#### Standardize the features using StandardScaler()

In [7]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler and Scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

### Define the Models and their hyperparameters
#### Models: [1] RandomForests, [2] Logistic, [3] XGBoost, [4] Gradient Boosting

In [ ]:
# Define models and their hyperparameter grids
models = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=78),
        'params': {
            'n_estimators': [100, 200, 500],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2]
        }
    },
    'LogisticRegression': {
        'model': LogisticRegression(random_state=78, max_iter=1000),
        'params': {
            'penalty': ['l1', 'l2', 'elasticnet', None],
            'C': [0.1, 1, 10, 100]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(random_state=78, use_label_encoder=False, eval_metric='logloss'),
        'params': {
            'n_estimators': [100, 200, 500],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2]
        }
    },
    'GradientBoosting': {
        'model': GradientBoostingClassifier(random_state=78),
        'params': {
            'n_estimators': [100, 200, 500],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2]
        }
    }
}

In [ ]:
# Train and evaluate each model
results = []

### Train and evaluate each model


In [ ]:
# Train and evaluate each model
results = []

In [ ]:
for model_name, model_info in models.items():
    print(f"Training {model_name}...")
    grid_search = GridSearchCV(
        estimator=model_info['model'],
        param_grid=model_info['params'],
        cv=3,  
        n_jobs=-1,
        scoring='accuracy',
        verbose=2
    )
    grid_search.fit(X_train_scaled, y_train)
    
    best_model = grid_search.best_estimator_
    
    # Evaluate on validation set
    val_predictions = best_model.predict(X_val_scaled)
    val_acc_score = accuracy_score(y_val, val_predictions)
    
    # Evaluate on test set
    test_predictions = best_model.predict(X_test_scaled)
    
    # Predictions and probabilities
    y_train_pred = best_model.predict(X_train_scaled)
    y_val_pred = best_model.predict(X_val_scaled)
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_train_prob = best_model.predict_proba(X_train_scaled)[:, 1]
    y_val_prob = best_model.predict_proba(X_val_scaled)[:, 1]
    y_test_prob = best_model.predict_proba(X_test_scaled)[:, 1]
    
    # Calculate losses
    train_loss = log_loss(y_train, y_train_prob)
    val_loss = log_loss(y_val, y_val_prob)
    test_loss = log_loss(y_test, y_test_prob)
    
    # Calculate errors
    train_error = 1 - accuracy_score(y_train, y_train_pred)
    val_error = 1 - accuracy_score(y_val, y_val_pred)
    test_error = 1 - accuracy_score(y_test, y_test_pred)
    
    # Confusion matrix and classification report
    cm = confusion_matrix(y_test, test_predictions)
    cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
    
    print(f"\n{model_name} Results")
    print("Confusion Matrix")
    print(cm_df)
    print(f"Accuracy Score: {accuracy_score(y_test, test_predictions)}")
    print("Classification Report")
    print(classification_report(y_test, test_predictions))
    
    results.append({
        'Model': model_name,
        'Best Params': grid_search.best_params_,
        'Validation Accuracy': val_acc_score,
        'Test Accuracy': accuracy_score(y_test, test_predictions),
        'Train Loss': train_loss,
        'Validation Loss': val_loss,
        'Test Loss': test_loss,
        'Train Error': train_error,
        'Validation Error': val_error,
        'Test Error': test_error
    })

In [ ]:
# Display results in a tabular format
results_df = pd.DataFrame(results)
print(tabulate(results_df, headers='keys', tablefmt='psql'))

In [ ]:
# Plotting model performance metrics
fig, axes = plt.subplots(2, 1, figsize=(12, 12))

# Plotting accuracy
results_df.plot(kind='bar', x='Model', y=['Validation Accuracy', 'Test Accuracy'], ax=axes[0], color=['blue', 'orange'])
axes[0].set_title('Model Accuracy Comparison')
axes[0].set_ylabel('Accuracy')
axes[0].set_xlabel('Model')

# Plotting loss
results_df.plot(kind='bar', x='Model', y=['Train Loss', 'Validation Loss', 'Test Loss'], ax=axes[1], color=['green', 'red', 'purple'])
axes[1].set_title('Model Loss Comparison')
axes[1].set_ylabel('Log Loss')
axes[1].set_xlabel('Model')

plt.tight_layout()
plt.show()

## 5.2. Under Sampling